In [1]:
import numpy as np
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing
import tensorflow as tf
import numpy as np
import tvm.relay as relay
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

import json
import numpy as np
import pygraphviz as pgv
PATH_MODEL = './model/'
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
            #A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(shape_size(json_data['attrs']['shape'][1][src[0]])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(shape_size(json_data['attrs']['shape'][1][node_idx])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

2022-05-03 17:54:07.489463: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [22]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))

input_data = np.random.normal(0,1,(1,512,512,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

In [ ]:
class JunoCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        self.pattern = is_tuple([self.var1, self.var2])
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 1
        self.target_node = []

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]
        var2 = node_map[self.var2][0]
        self.target_node.append(var2)
        # return post
        # return post
        # original = post
        if self.cnt != self.target:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(var2, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            print(type(var1))
            print(type(cast_to_float32))
            print(type(post))
            return relay.op.Tuple([var1, cast_to_float32])



In [2]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("resnet152", 224))

input_data = np.random.normal(0,1,(1,224,224,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


2022-05-03 17:54:18.694850: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-03 17:54:18.697916: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-03 17:54:18.813208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 17:54:18.816774: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-03 17:54:18.816881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-03 17:54:18.877416: I tensorflow/stream_executor/platfor

In [3]:
mod

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 224, 224), float32], %v_param_1: Tensor[(64, 3, 7, 7), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_19: Tensor[(256, 64, 1, 1), float32], %v_param_20: Tensor[(256), float32], %v_param_23: Tensor[(256), float32], %v_param_24: Tensor[(256), float32], %v_param_25: Tensor[(256), float32], %v_param_26: Tensor[(256), float32], %v_param_7: Tensor[(64, 64, 1, 1), float32], %v_param_8: Tensor[(64), float32], %v_param_9: Tensor[(64), float32], %v_param_10: Tensor[(64), float32], %v_param_11: Tensor[(64), float32], %v_param_12: Tensor[(64), float32], %v_param_13: Tensor[(64, 64, 3, 3), float32], %v_param_14: Tensor[(64), float32], %v_param_15: Tensor[(64), float32], %v_param_16: Tensor[(64), float32], %v_param_17: Tensor[(64), float32], %v_param_18: Tensor[(64), float32], %v_param_21: Tensor[(256

In [13]:
# pattern = is_op("nn.relu")(is_tuple_get_item(is_op('nn.batch_norm')))
conv2d_1 = is_op("nn.conv2d")(wildcard(), wildcard())
bias_add_1 = is_op("nn.bias_add")(conv2d_1, wildcard())
pattern =  bias_add_1
# %60 = nn.conv2d(%59, %v_param_79, strides=[2, 2], padding=[0, 0, 0, 0], channels=512, kernel_size=[1, 1]);
# %61 = nn.bias_add(%60, %v_param_80);
# %62 = nn.batch_norm(%61, %v_param_83, %v_param_84, %v_param_85, %v_param_86, epsilon=1.001e-05f);
# %63 = nn.conv2d(%59, %v_param_67, strides=[2, 2], padding=[0, 0, 0, 0], channels=128, kernel_size=[1, 1]);
# %64 = nn.bias_add(%63, %v_param_68);

partitioned = pattern.partition(mod['main'])

In [91]:
x = relay.var("x", shape=(8, 8))
y = relay.var("y", shape=(8, 8))
z = relay.var("z", shape=(8, 8))
k = relay.var("k", shape=(8, 8))

p = (x+y)*z + k
a = (x+y)
b = a * z + k
c = b - a


In [92]:
f = relay.Function([x,y,z,k], c)

In [93]:
f

fn (%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32], %z: Tensor[(8, 8), float32], %k: Tensor[(8, 8), float32]) {
  %0 = add(%x, %y);
  %1 = multiply(%0, %z);
  %2 = add(%1, %k);
  subtract(%2, %0)
}

In [94]:
f1 = relay.Function()

TypeError: __init__() missing 2 required positional arguments: 'params' and 'body'

In [95]:
f.body.

Op(subtract)

In [109]:
l = tvm.relay.var

In [105]:
v = f.body.args[1].args[0]

In [114]:
print(f.body.args[1])

free_var %x: Tensor[(8, 8), float32];
free_var %y: Tensor[(8, 8), float32];
add(%x, %y)


In [112]:
tvm.relay.shape_of(v)

CallNode(Op(shape_of), [Var(x, ty=TensorType([8, 8], float32))], relay.attrs.ShapeOfAttrs(0x21dc34e8), [])

In [ ]:
def visit(node):
    print(node)

In [116]:
tvm.tir.stmt_functor.post_order_visit(mod)

TypeError: post_order_visit() missing 1 required positional argument: 'fvisit'

In [115]:
f.body.args

2

In [53]:
class FuncCollector(tvm.relay.ExprVisitor):
    def __init__(self):
        super().__init__()
        # self.funcs = set()
        self.funcs = []

    def visit_call(self, call):
        # print(call)
        # op = call.op
        op = call
        print(type(call))
        if isinstance(op, relay.Call):
            self.funcs.append(op)

collector = FuncCollector()
collector.visit(f)
print("Functions:")
for func in collector.funcs:
    print(func)

<class 'tvm.relay.expr.Call'>
Functions:
free_var %x: Tensor[(8, 8), float32];
free_var %y: Tensor[(8, 8), float32];
%0 = add(%x, %y);
free_var %z: Tensor[(8, 8), float32];
%1 = multiply(%0, %z);
free_var %k: Tensor[(8, 8), float32];
add(%1, %k)


In [65]:
# ev = tvm.relay.ExprVisitor()
ev = FuncCollector()
ev.visit(f)

1


In [64]:
class FuncCollector(tvm.relay.ExprVisitor):
    def __init__(self):
        super().__init__()
        # self.funcs = set()
        self.funcs = []

    def visit_call(self, call):
        # print(call)
        # op = call.op
        print(1)
        # op = call
        # print(type(call))
        # if isinstance(op, relay.Call):
        #     self.funcs.append(op)


In [68]:
a  =f.body.args[0].args[0].op

In [70]:
relay.op(a)

TypeError: 'module' object is not callable

Op(nn.softmax)

In [17]:
tvm.relay.transform.PartitionGraph()(partitioned)

TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::transform::Pass, tvm::IRModule)>::AssignTypedLambda<tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}>(tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::IRModule<tvm::IRModule>() const
  3: TVMFuncCall
  2: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::transform::Pass, tvm::IRModule)>::AssignTypedLambda<tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}>(tvm::transfo

In [3]:
mod

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 224, 224), float32], %v_param_1: Tensor[(64, 3, 7, 7), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_19: Tensor[(256, 64, 1, 1), float32], %v_param_20: Tensor[(256), float32], %v_param_23: Tensor[(256), float32], %v_param_24: Tensor[(256), float32], %v_param_25: Tensor[(256), float32], %v_param_26: Tensor[(256), float32], %v_param_7: Tensor[(64, 64, 1, 1), float32], %v_param_8: Tensor[(64), float32], %v_param_9: Tensor[(64), float32], %v_param_10: Tensor[(64), float32], %v_param_11: Tensor[(64), float32], %v_param_12: Tensor[(64), float32], %v_param_13: Tensor[(64, 64, 3, 3), float32], %v_param_14: Tensor[(64), float32], %v_param_15: Tensor[(64), float32], %v_param_16: Tensor[(64), float32], %v_param_17: Tensor[(64), float32], %v_param_18: Tensor[(64), float32], %v_param_21: Tensor[(256

In [15]:
class Resnet152Callback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        # self.pattern = is_tuple([self.var1, self.var2])
        bn_node = is_op('nn.batch_norm')(wildcard(), wildcard(), wildcard(), wildcard(), wildcard())
        tuple_get_item_node = is_tuple_get_item(bn_node, 0)
        add_node = is_op('add')(wildcard(), tuple_get_item_node)
        relu_node = is_op('nn.relu')(add_node)
        self.pattern = relu_node
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 1
        self.target_node = []

    def callback(self, pre, post, node_map):
        # if self.cnt != self.target:
        if self.cnt % 3 != 0 or self.cnt / 3 > 5:
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(8.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.divide(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='float32'), relay.const(8.0))

            # cast_to_float32 = relay.cast(
            #     relay.clip(
            #         relay.right_shift(
            #             relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
            #             relay.const(10)),
            #         a_min=-127.0, a_max=127.0), 
            #     dtype="float32"
            # )
            return cast_to_float32



In [26]:
class UnetPreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.var2 = wildcard()
        tuple_node = is_tuple([wildcard(), self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        self.match_node = []

    def callback(self, pre, post, node_map):
        var2 = node_map[self.var2][0]
        self.match_node.append(var2)
        return post


class UnetCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()

        # self.batch_norm_node = is_op('nn.batch_norm')(wildcard())
        # self.pattern_1 = self.batch_norm_node

        self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        self.pattern_1 = self.tuple_get_item_node

        self.max_pool2d_arg_node = wildcard() 
        self.max_pool2d_node = is_op('nn.max_pool2d')(self.max_pool2d_arg_node)
        self.pattern_2 = self.max_pool2d_node

        self.concat_arg_1 = wildcard()
        self.concat_arg_2 = wildcard()
        tuple_node = is_tuple([self.concat_arg_1, self.concat_arg_2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern_3 = concat_node

        self.pattern = (self.pattern_1 | self.pattern_2 | self.pattern_3)
        self.match_node = match_node

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(16.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        return relay.annotation.stop_fusion(cast_to_int8)

    def dequant(self, node):
        cast_to_float32 = relay.cast(
            relay.clip(
                relay.right_shift(
                    relay.add(relay.cast(node, dtype='int32'), relay.const(512)),
                    relay.const(10)),
                a_min=-127.0, a_max=127.0), 
            dtype="float32"
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern_1.match(pre):
            # print("1")
            if pre in self.match_node:
                print("pat 1")
                return self.quant(post)
        elif self.pattern_2.match(pre):
            # print("2")
            max_pool2d_arg_node = node_map[self.max_pool2d_arg_node][0]
            # if max_pool2d_arg_node in self.match_node:
            if True:
                print("pat 2")
                return relay.nn.max_pool2d(self.dequant(max_pool2d_arg_node), pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
        elif self.pattern_3.match(pre):
            # print("3")
            concat_arg_1 = node_map[self.concat_arg_1][0]
            concat_arg_2 = node_map[self.concat_arg_2][0]
            # if concat_arg_2 in self.match_node:
            if True:
                print("pat 3")
                return relay.concatenate((concat_arg_1, self.dequant(concat_arg_2)), axis=1)
        else:
            print("asd")
        return post

SyntaxError: invalid syntax (1858775713.py, line 73)

In [38]:
class UnetPreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.var2 = wildcard()
        tuple_node = is_tuple([wildcard(), self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        self.match_node = []

    def callback(self, pre, post, node_map):
        var2 = node_map[self.var2][0]
        self.match_node.append(var2)
        return post


class UnetCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        self.pattern_1 = self.tuple_get_item_node

        self.pattern = self.pattern_1
        self.match_node = match_node

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(8.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        return relay.annotation.stop_fusion(cast_to_int8)

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(8.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern_1.match(pre):
            if pre in self.match_node:
                print("pat 1")
                return self.dequant(self.quant(post))
        return post

In [39]:
upc = UnetPreProcessCallback()
rewrite(upc, mod['main'])
uc = UnetCallback(upc.match_node)
out = rewrite(uc, mod['main'])

pat 1
pat 1
pat 1
pat 1


In [4]:
class Resnet152PreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.var1 = wildcard()
        self.var2 = wildcard()
        tuple_node = is_tuple([self.var1, self.var2])
        concat_node = is_op('add')(tuple_node)
        self.pattern = concat_node
        self.match_node = []
        self.counter = 0
        self.slice_limit = 10

    def callback(self, pre, post, node_map):
        if self.counter < self.slice_limit:
            var2 = node_map[self.var2][0]
            self.match_node.append(var2)
            self.counter += 1
        return post


class Resnet152Callback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        self.pattern_1 = self.tuple_get_item_node

        self.pattern = self.pattern_1
        self.match_node = match_node

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(8.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        return relay.annotation.stop_fusion(cast_to_int8)

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(8.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern_1.match(pre):
            if pre in self.match_node:
                print("pat 1")
                return self.dequant(self.quant(post))
        return post

In [40]:
out

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [ ]:
class UnetPreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        tuple_node = is_tuple([self.var1, self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        self.target_node = []

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]
        var2 = node_map[self.var2][0]
        self.target_node.append(var2)
        cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(var2), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
        )
        return relay.concatenate([var1, cast_to_float32], axis=1)

In [4]:
class UnetPreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        tuple_node = is_tuple([self.var1, self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        self.target_node = []

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]
        var2 = node_map[self.var2][0]
        self.target_node.append(var2)
        cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(var2), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
        )
        return relay.concatenate([var1, cast_to_float32], axis=1)


class UnetMidProcessCallback(DFPatternCallback):
    def __init__(self, target_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        self.var1 = wildcard()
        self.var2 = wildcard()
        tuple_node = is_tuple_get_item(wildcard(), 0)
        self.pattern = tuple_node
        self.target_node = target_node
        self.target_node_2 = []

    def callback(self, pre, post, node_map):
        if pre in self.target_node:
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            self.target_node_2.append(cast_to_int8)
            return cast_to_int8
        else:
            return post


class UnetCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, target_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        max_pool_node = is_op('nn.max_pool2d')(self.var1)
        self.pattern = max_pool_node
        self.cnt = 0
        self.target = 1
        self.target_node = target_node

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]

        # if var1 in self.target_node:
        if True:
            print("sex")
            # cast_to_int8 = relay.cast(
            #     relay.clip(
            #         relay.round(
            #             relay.multiply(var1, relay.const(16.0))
            #         ), 
            #         a_min=-127.0, a_max=127.0
            #     ),
            #     dtype="int8"
            # )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(var1), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32
        else:
            return post




In [5]:
upp = UnetPreProcessCallback()
out = rewrite(upp, mod['main'])
print("==============")
print(upp.target_node)

[TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.leaky_relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.leaky_relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.leaky_relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.leaky_relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1,

In [227]:
upp = UnetPreProcessCallback()
out = rewrite(upp, mod['main'])
print(out)
print("==============")

# out = mod['main']
ump = UnetMidProcessCallback(upp.target_node)
out = rewrite(ump, out)

print(out)
print("==============")
# ump = UnetMidProcessCallback(upp.target_node)
# out = rewrite(ump, out)
# upp = UnetPreProcessCallback()
# rewrite(upp, out)
print("==============")

out = rewrite(UnetCallback(ump.target_node_2), out)
print(out)


# for target_node in list(reversed(upp.target_node)):
    # out = rewrite(UnetCallback([target_node]), out)

# print(out)

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [16]:
out = rewrite(Resnet152Callback(), mod['main'])
print(out)

fn (%input_1: Tensor[(1, 3, 224, 224), float32], %v_param_1: Tensor[(64, 3, 7, 7), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_19: Tensor[(256, 64, 1, 1), float32], %v_param_20: Tensor[(256), float32], %v_param_23: Tensor[(256), float32], %v_param_24: Tensor[(256), float32], %v_param_25: Tensor[(256), float32], %v_param_26: Tensor[(256), float32], %v_param_7: Tensor[(64, 64, 1, 1), float32], %v_param_8: Tensor[(64), float32], %v_param_9: Tensor[(64), float32], %v_param_10: Tensor[(64), float32], %v_param_11: Tensor[(64), float32], %v_param_12: Tensor[(64), float32], %v_param_13: Tensor[(64, 64, 3, 3), float32], %v_param_14: Tensor[(64), float32], %v_param_15: Tensor[(64), float32], %v_param_16: Tensor[(64), float32], %v_param_17: Tensor[(64), float32], %v_param_18: Tensor[(64), float32], %v_param_21: Tensor[(256, 64, 1, 1), float32], %v_p

In [21]:
# class JunoCallback(DFPatternCallback):
#     # A callback class to rewrite the matched pattern to a batch_norm op.
#     def __init__(self, require_type=False):
#         super().__init__(require_type)
#         super().__init__(rewrite_once=True)
#         # self.x = wildcard()
#         self.var1 = wildcard()
#         self.var2 = wildcard()
#         self.pool_size = is_constant()
#         self.strides = is_constant()
#         self.padding = is_constant()
#         self.padding = [1, 1, 1, 1]
#         bn_node = is_op('nn.batch_norm')(wildcard(), wildcard(), wildcard(), wildcard(), wildcard())
#         tuple_get_item_node = is_tuple_get_item(bn_node, 0)
#         tupling = is_tuple([wildcard(), self.var2])
#         self.pattern = dominates(tuple_get_item_node, wildcard(), tupling)
#         # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
#         self.cnt = 0
#         self.target = 1
#         self.target_node = []

#     def callback(self, pre, post, node_map):
#         # var1 = node_map[self.var1][0]
#         var2 = node_map[self.var2][0]
#         # self.target_node.append(var2)
#         print(post)        # return post
#         # return post
#         # original = post
#         if self.cnt != self.target:
#             self.cnt += 1
#             return post
#         else:
#             self.cnt += 1
#             cast_to_int8 = relay.cast(
#                 relay.clip(
#                     relay.round(
#                         relay.multiply(post, relay.const(16.0))
#                     ), 
#                     a_min=-127.0, a_max=127.0
#                 ),
#                 dtype="int8"
#             )
#             cast_to_float32 = relay.cast(
#                 relay.clip(
#                     relay.right_shift(
#                         relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
#                         relay.const(10)),
#                     a_min=-127.0, a_max=127.0), 
#                 dtype="float32"
#             )
#             print(type(var1))
#             print(type(cast_to_float32))
#             print(type(post))
#             return relay.op.Tuple([var1, cast_to_float32])



In [23]:
jc = JunoCallback()
out = rewrite(jc, mod['main'])

In [10]:
class Juno2Callback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, target_pat, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        self.padding = [1, 1, 1, 1]
        self.pattern = TupleGetItemPattern(target_pat)
        # self.pattern = is_op('nn.batch_norm')(wildcard(), wildcard(),wildcard(),wildcard(),wildcard())
        self.cnt = 0
        self.target = 1

    def callback(self, pre, post, node_map):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(post, relay.const(16.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        cast_to_float32 = relay.cast(
            relay.clip(
                relay.right_shift(
                    relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                    relay.const(10)),
                a_min=-127.0, a_max=127.0), 
            dtype="float32"
        )
        print(type(cast_to_float32))
        print(type(post))
        return cast_to_float32

In [11]:
jc2 = Juno2Callback(jc.target_node[0])

TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::relay::TupleGetItemPattern (tvm::relay::DFPattern, int)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, int)#26}>(tvm::relay::{lambda(tvm::relay::DFPattern, int)#26}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::relay::DFPattern<tvm::relay::DFPattern>() const
  3: TVMFuncCall
  2: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::relay::TupleGetItemPattern (tvm::relay::DFPattern, int)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, int)#26}>(tvm::relay::{l

In [9]:
out = rewrite(jc2, mod['main'])

TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: tvm::runtime::TypedPackedFunc<tvm::relay::DFPatternCallback (tvm::relay::DFPattern, tvm::runtime::PackedFunc, bool, bool)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, tvm::runtime::PackedFunc, bool, bool)#2}>(tvm::relay::{lambda(tvm::relay::DFPattern, tvm::runtime::PackedFunc, bool, bool)#2}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const, tvm::runtime::TVMRetValue) const
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::relay::DFPattern<tvm::relay::DFPattern>() const
  3: TVMFuncCall
  2: tvm::runtime::TypedPackedFunc<tvm::relay::DFPatternCallback (tvm::relay::DFPattern, tvm::runtime::PackedFunc, bool, bool)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, tvm::runtime::PackedFunc, bool, bool)#2}>(tvm::relay::{lambda(tvm::relay::DF

In [4]:
model_keras = tf.keras.models.load_model(PATH_MODEL + '{}_{}.h5'.format("resnet152", 224))

input_data = (np.random.randint(0,256,(1,224,224,3)) / 255).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

2022-04-24 11:25:55.544997: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-24 11:25:55.548184: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-24 11:25:55.579905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 11:25:55.583478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-24 11:25:55.583497: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-24 11:25:55.646990: I tensorflow/stream_executor/platfor

In [4]:
print(mod)

def @main(%input_1: Tensor[(1, 3, 224, 224), float32], %v_param_1: Tensor[(64, 3, 7, 7), float32], %v_param_2: Tensor[(64), float32], %v_param_3: Tensor[(64), float32], %v_param_4: Tensor[(64), float32], %v_param_5: Tensor[(64), float32], %v_param_6: Tensor[(64), float32], %v_param_19: Tensor[(256, 64, 1, 1), float32], %v_param_20: Tensor[(256), float32], %v_param_23: Tensor[(256), float32], %v_param_24: Tensor[(256), float32], %v_param_25: Tensor[(256), float32], %v_param_26: Tensor[(256), float32], %v_param_7: Tensor[(64, 64, 1, 1), float32], %v_param_8: Tensor[(64), float32], %v_param_9: Tensor[(64), float32], %v_param_10: Tensor[(64), float32], %v_param_11: Tensor[(64), float32], %v_param_12: Tensor[(64), float32], %v_param_13: Tensor[(64, 64, 3, 3), float32], %v_param_14: Tensor[(64), float32], %v_param_15: Tensor[(64), float32], %v_param_16: Tensor[(64), float32], %v_param_17: Tensor[(64), float32], %v_param_18: Tensor[(64), float32], %v_param_21: Tensor[(256, 64, 1, 1), float32]

In [25]:
out

fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_param_26

In [41]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(out, target, params=params)


In [42]:
lib.export_library("./model/unet_cuda_try_quant.so")
# lib.export_library("./model/resnet152_cuda_try_quant.so")


In [75]:
show_graph(lib['get_graph_json'](), "unet_insert_quant")

In [48]:
def show_graph_partition(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            dtype = json_data['attrs']['dltype'][1][src[0]]
            if dtype == 'int8':
                n = A.get_node(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
                n.attr["color"] = "green"

    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

In [49]:
show_graph_partition(lib['get_graph_json'](), "unet_insert_quant_colored")